In [2]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.pipeline import make_pipeline
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import pickle
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
from sklearn.externals import joblib 


In [3]:
#path1 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Preprocessing/hurto_motos_categorias_ingCaracteristicas.csv"
path2 = "https://raw.githubusercontent.com/jaarciah/Proyecto-Integrador-2020-periodo-2/master/Data/Modeling/hurto_preprocesado_C.csv"

#hurtos_moto = pd.read_csv(path1, sep=";", low_memory=False)
hurto_preprocesado = pd.read_csv(path2, sep=",", low_memory=False)

In [4]:
hurto_preprocesado.modalidad.value_counts()

0    33100
1    25455
Name: modalidad, dtype: int64

In [5]:
hurto_preprocesado.head()

,latitud,longitud,estado_civil,medio_transporte,modalidad,nombre_barrio,codigo_comuna,lugar,sede_receptora,modelo,...,anho,mes,dia,dia_semana,hora,festivos,quincena,week_number,ferias_fiestas,franja_horaria
0,6.268040,-75.549892,0,0,0,0,0,0,0,0,...,2017,1,1,6,23,1,1,52,0,2
1,6.268129,-75.557882,0,0,0,4,3,2,3,4,...,2017,1,1,6,16,1,1,52,0,1
2,6.268399,-75.549576,0,0,0,0,0,0,0,4,...,2017,1,1,6,10,1,1,52,0,0
3,6.263576,-75.558601,3,0,0,4,3,0,3,4,...,2017,1,3,1,8,0,0,1,0,0
4,6.263299,-75.556534,0,1,1,4,3,0,3,0,...,2017,1,7,5,3,0,0,1,0,2


In [6]:
hurto_preprocesado.columns

Index(['latitud', 'longitud', 'estado_civil', 'medio_transporte', 'modalidad',
       'nombre_barrio', 'codigo_comuna', 'lugar', 'sede_receptora', 'modelo',
       'fecha', 'anho', 'mes', 'dia', 'dia_semana', 'hora', 'festivos',
       'quincena', 'week_number', 'ferias_fiestas', 'franja_horaria'],
      dtype='object')

In [7]:
# variables escogidas despues del proceso de ingenieria de caracteristica.
variables = ["latitud","longitud","estado_civil","medio_transporte","modalidad","nombre_barrio","codigo_comuna","lugar","sede_receptora","modelo", "fecha"]

hurto_preprocesado_new = hurto_preprocesado[variables]

# Proceso de cross-validation.

In [8]:
y = hurto_preprocesado_new.modalidad
x = hurto_preprocesado_new.drop(['modalidad'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=8)

In [9]:
def class_report(y_true, y_pred, nombre_clasificador):
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import precision_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import jaccard_score
    
    TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()
    
    Acurracy = ((TP + TN) / (TP + FN + FP + TN))
    Recall = (TP / (TP + FN))
    Precision = precision_score(y_test, ypred)
    Specificity = (TN / (FP + TN))
    F1 = f1_score(y_test, ypred)
    
    result = [nombre_clasificador, Acurracy, Recall, Precision, Specificity, F1]
    
    return result

# 2. Segunda iteración.

## Resultados:

In [10]:
nameModel = ["modelRLB.pkl", "modelSvm.pkl", "modelSGD.pkl", "modelNN.pkl", "modelGNB.pkl", "modelDTre.pkl", "modelETre.pkl", "modelbaggingDTre.pkl",
          "modelbaggingNN.pkl", "modelbaggingETre.pkl", "modelRF.pkl", "modelAdaB.pkl", "modelGTreB.pkl"]

Model = ["Regresión logistica", "SVM", "Stochastic Gradient Descent", "KNN", "Naive Bayes", "Decision Tree", " ExtraTreeClassifier",
        "Bagging C Decision Tree", "Bagging C KNN", "Bagging C Extra Tree", "RandomFores", "AdaBoost", "Gradient Tree Boosting"]

total = []
for id_nameModel, valor_nameModel in enumerate(nameModel):
    # Save to file in the current working directory
    joblib_file = valor_nameModel
    # Load from file
    model = joblib.load(joblib_file)
    ypred = model.predict(X_test)
    report = class_report(y_test, ypred, Model[id_nameModel])
    total.append(report)

primera_result = pd.DataFrame(total)
primera_result = primera_result.rename(columns={0: "Clasificador", 1: "Acurracy", 2: "Recall", 3: "Precision", 4: "Specificity", 5: "F1"})
primera_result

,Clasificador,Acurracy,Recall,Precision,Specificity,F1
0,Regresión logistica,0.568440,0.000000,0.000000,1.000000,0.000000
1,SVM,0.568440,0.000000,0.000000,1.000000,0.000000
2,Stochastic Gradient Descent,0.568440,0.000000,0.000000,1.000000,0.000000
3,KNN,0.621552,0.575782,0.559831,0.656302,0.567694
4,Naive Bayes,0.612416,0.664424,0.541526,0.572931,0.596713
5,Decision Tree,0.865682,0.852394,0.838948,0.875770,0.845618
6,ExtraTreeClassifier,0.853386,0.837752,0.825180,0.865255,0.831419
7,Bagging C Decision Tree,0.895654,0.870202,0.885979,0.914977,0.878020
8,Bagging C KNN,0.616429,0.554214,0.555754,0.663662,0.554983
9,Bagging C Extra Tree,0.892580,0.866245,0.882661,0.912573,0.874376


In [39]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=go.Scatter(x=primera_result['Clasificador'],
                                y=primera_result['Acurracy'],
                                text = "Acurracy",
                               name = "Acurracy"))

fig.add_trace(go.Scatter(x=primera_result['Clasificador'],
                                y=primera_result['Recall'],
                        text = "Recall",
                        name = "Recall"))

fig.update_layout(title='Metricas de evaluación modelos de clasificación')
fig.show()

In [40]:
fig = go.Figure(data=go.Scatter(x=primera_result['Clasificador'],
                                y=primera_result['Precision'],
                                text = "Precision",
                               name = "Precision"))

fig.add_trace(go.Scatter(x=primera_result['Clasificador'],
                                y=primera_result['Specificity'],
                        text = "Specificity",
                        name = "Specificity"))

fig.update_layout(title='Metricas de evaluación modelos de clasificación', )
fig.show()

In [41]:
fig = go.Figure(data=go.Scatter(x=primera_result['Clasificador'],
                                y=primera_result['F1'],
                                text = "F1",
                               name = "F1"))

fig.update_layout(title='Metricas de evaluación modelos de clasificación', )
fig.show()